*https://github.com/Aubrey-Bermuda-CA/IFRS9-MODELING-CHINA*

*3.2 Function description: Read macro historical data and external prediction data, use HP filter to filter the data, and use autoarima to predict macro data.*

In [1]:
import time
import pandas as pd
import pmdarima as pm
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import STL

In [2]:
# set date range
start_date = '2010-03-31'
forecast_date = '2026-09-30'
date_index = pd.date_range(start=start_date, end=forecast_date, freq='Q')

# read macro data
macro_his_data = pd.read_csv(f'./macrofactor_historical_data.csv', index_col=[0], parse_dates=[0])
macro_fc_data = pd.read_csv(f'./macrofactor_forecast_data.csv', index_col=[0], parse_dates=[0])

# combine data
macro_data = macro_his_data.combine_first(macro_fc_data)

# Missing values ​​can be handled using interpolation. There are several methods to choose from:
# 'linear', 'nearest', 'zero', 'slinear', 'quadratic', 'cubic', 'barycentric', 'polynomial', 
# 'krogh', 'piecewise_polynomial', 'spline', 'pchip', 'akima', 'cubicspline'
# macro_data = macro_data.interpolate(method='spline')

macro_data

,GDP_Const_Qtr,Ind_Pro_Mth,CPI_Mth,Core_CPI_Mth,PPI_Mth,FAI_Cum,Retail_Sales_Mth,Imports_Mth,Exports_Mth,M1,...,PPIRM_Mth,CGPI_Mth,70_Cities_Price_Mth,Power_Gen_Mth,Freight_Mth,Urban_Income_Cum,Urban_Exp_Cum,Nat_Housing_Index,Macro_Eco_Index,Housing_Sales_Cum
date,,,,,,,,,,,,,,,,,,,,,
2010-03-31,12.20,18.1,102.400,NaN,5.91,26.4,18.00,66.27,24.21,29.94,...,11.46,105.6,9.5,17.6,15.800000,9.8,11.01,105.89,104.0,57.7
2010-06-30,10.80,13.7,102.900,NaN,6.41,25.5,18.30,33.87,43.87,24.56,...,10.80,106.6,7.7,11.4,12.300000,10.2,9.89,105.06,102.6,25.4
2010-09-30,9.90,13.3,103.600,NaN,4.33,24.5,18.84,24.38,25.08,20.87,...,7.10,106.1,6.2,8.1,11.900000,10.5,9.32,103.52,101.9,15.9
2010-12-31,9.90,13.5,104.600,NaN,5.93,24.5,19.10,25.94,17.87,21.19,...,9.47,107.9,5.0,5.1,0.310000,11.3,9.84,101.79,103.6,18.3
2011-03-31,10.20,14.8,105.383,NaN,7.31,25.0,17.40,27.50,35.76,15.00,...,10.53,109.3,3.7,14.8,14.300000,12.3,10.69,102.98,102.5,27.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-30,4.60,5.4,100.400,0.1,-2.80,3.4,3.20,0.30,2.40,-7.40,...,-2.20,98.3,-9.0,6.0,2.315156,4.5,5.00,92.40,NaN,-22.7
2024-12-31,4.85,NaN,1.800,0.7,-2.20,3.0,6.40,11.00,NaN,-6.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-03-31,4.00,NaN,NaN,NaN,NaN,3.6,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# The method used here comes from this article ：https://www.ecb.europa.eu/pub/pdf/scpwps/ecbwp499.pdf. 
# This is just a simple attempt, not a reproduction of the article.
# As concluded in the article, hpfilter obtain more stable predictive values.

# Use Hodrick-Prescott filter to separate trend terms and periodic terms
macro_data_fitted = pd.DataFrame(index=macro_data.index)

for column in macro_data:
    y = macro_data[column].dropna()
    cycle, trend = sm.tsa.filters.hpfilter(y)
    try:
        result = STL(cycle).fit()
        seasonal = result.seasonal
        seasonal = seasonal.rename(column)
        seasonal_smooth = seasonal[abs(seasonal - seasonal.mean()) <= 2.576 * seasonal.std()] # 99% confidence level value
        fitted = trend.add(seasonal_smooth, fill_value=0)
    except:
        seasonal = pd.DataFrame(index=trend.index, columns=[column])
        cycle_smooth = cycle[abs(cycle - cycle.mean()) <= 2.576 * cycle.std()]
        fitted = trend.add(cycle_smooth, fill_value=0)
    
    # # Create a graphics object
    # fig, ax1 = plt.subplots(figsize=(10, 5))
    # ax1.plot(y.index, y, label=column, color='g')
    # ax1.plot(y.index, fitted, label='Fitted', color='b')
    # ax1.set_xlabel('Date')
    # ax1.set_ylabel('Value and Fitted')
    # ax1.legend(loc='upper left')

    # # Create a secondary axis
    # ax2 = ax1.twinx() 
    # ax2.plot(y.index, seasonal, label='Seasonal', color='gray', linestyle='--')
    # ax2.set_ylabel('Seasonal', color='k')
    # ax2.legend(loc='upper right')
    # plt.title('Original, Seasonal and Fitted (HP filter)')
    # plt.show()

    macro_data_fitted = macro_data_fitted.merge(fitted.rename(column), left_index=True, right_index=True, how='left')

macro_data_fitted.to_csv(f'./3.2_macro_data_fitted.csv')

In [4]:
# macrofactor forecast with autoarima
df = macro_data_fitted
predict_data = pd.DataFrame(index=date_index)
print(f'macrofoctor_list：{df.columns.values}')

# timer
start_t = time.perf_counter()

# fitmodel
for column in df:
    y = df[column].dropna()
    # seasonal data use 4
    m = 4
    # calc d and D firt to save time
    d = pm.arima.ndiffs(y, alpha=0.05, test='adf', max_d=2)
    D = pm.arima.nsdiffs(y, m=m, test='ocsb', max_D=2)
    print(f'Traverse {column} equation...')
    arima = pm.auto_arima(y, m=m, method='lbfgs', # 'newton', 'nm', 'bfgs', 'lbfgs', 'powell', 'cg', 'ncg', 'basinhopping'
                          start_p=1, start_q=1, d=d, 
                          start_P=1, start_Q=1, D=D,
                          max_p=5, max_q=5, 
                          max_P=5, max_Q=5,
                          seasonal=True, alpha=0.05,
                          information_criterion='aic', # 'aic', 'bic', 'hqic', 'oob'
                          stepwise=True, # The stepwise algorithm can be significantly faster than fitting all
                          error_action='ignore', 
                          suppress_warnings=True,
                          trace=True
                          )
    # predict
    pr_in = arima.predict_in_sample(X=None, dynamic=False, return_conf_int=False)
    pr_out = arima.predict(n_periods=15, X=None, return_conf_int=False)
    pr = pd.DataFrame({column:pd.concat([pr_in, pr_out])})
    predict_data = predict_data.merge(pr, left_index=True, right_index=True, how='left')

# Output prediction results
predict_data = macro_data.combine_first(predict_data) # This does not change the value of historical data
# predict_data = macro_data_fitted.combine_first(predict_data) # If you want to use filtered historical values
predict_data.to_csv(f'./3.2_predict_data.csv')

dur = (time.perf_counter() - start_t) / 60
print(f'The prediction process took a total of {dur:,.2f} minutes.')

macrofoctor_list：['GDP_Const_Qtr' 'Ind_Pro_Mth' 'CPI_Mth' 'Core_CPI_Mth' 'PPI_Mth'
 'FAI_Cum' 'Retail_Sales_Mth' 'Imports_Mth' 'Exports_Mth' 'M1' 'M2'
 'Loans_RMB' 'Social_Financing' 'Manuf_PMI' 'Ind_Revenue_Cum' 'PPIRM_Mth'
 'CGPI_Mth' '70_Cities_Price_Mth' 'Power_Gen_Mth' 'Freight_Mth'
 'Urban_Income_Cum' 'Urban_Exp_Cum' 'Nat_Housing_Index' 'Macro_Eco_Index'
 'Housing_Sales_Cum']
Traverse GDP_Const_Qtr equation...
Performing stepwise search to minimize aic
 ARIMA(1,1,1)(1,0,1)[4] intercept   : AIC=69.383, Time=0.10 sec
 ARIMA(0,1,0)(0,0,0)[4] intercept   : AIC=95.085, Time=0.01 sec
 ARIMA(1,1,0)(1,0,0)[4] intercept   : AIC=75.587, Time=0.04 sec
 ARIMA(0,1,1)(0,0,1)[4] intercept   : AIC=69.020, Time=0.04 sec
 ARIMA(0,1,0)(0,0,0)[4]             : AIC=95.928, Time=0.01 sec
 ARIMA(0,1,1)(0,0,0)[4] intercept   : AIC=77.418, Time=0.03 sec
 ARIMA(0,1,1)(1,0,1)[4] intercept   : AIC=67.452, Time=0.08 sec
 ARIMA(0,1,1)(1,0,0)[4] intercept   : AIC=65.641, Time=0.05 sec
 ARIMA(0,1,1)(2,0,0)[4] i

C:\Program Files\Python39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Program Files\Python39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


 ARIMA(0,0,0)(0,0,0)[4] intercept   : AIC=163.121, Time=0.02 sec
 ARIMA(1,0,0)(1,0,0)[4] intercept   : AIC=118.455, Time=0.12 sec
 ARIMA(0,0,1)(0,0,1)[4] intercept   : AIC=inf, Time=0.09 sec
 ARIMA(0,0,0)(0,0,0)[4]             : AIC=633.142, Time=0.01 sec
 ARIMA(1,0,0)(0,0,0)[4] intercept   : AIC=139.317, Time=0.06 sec
 ARIMA(1,0,0)(2,0,0)[4] intercept   : AIC=inf, Time=0.22 sec
 ARIMA(1,0,0)(1,0,1)[4] intercept   : AIC=83.898, Time=0.19 sec
 ARIMA(1,0,0)(0,0,1)[4] intercept   : AIC=inf, Time=0.12 sec
 ARIMA(1,0,0)(2,0,1)[4] intercept   : AIC=inf, Time=nan sec
 ARIMA(1,0,0)(1,0,2)[4] intercept   : AIC=inf, Time=0.26 sec
 ARIMA(1,0,0)(0,0,2)[4] intercept   : AIC=inf, Time=0.18 sec
 ARIMA(1,0,0)(2,0,2)[4] intercept   : AIC=inf, Time=0.25 sec
 ARIMA(0,0,0)(1,0,1)[4] intercept   : AIC=inf, Time=0.13 sec
 ARIMA(2,0,0)(1,0,1)[4] intercept   : AIC=inf, Time=0.20 sec
 ARIMA(0,0,1)(1,0,1)[4] intercept   : AIC=inf, Time=0.16 sec
 ARIMA(2,0,1)(1,0,1)[4] intercept   : AIC=inf, Time=0.22 sec
 ARIMA

C:\Program Files\Python39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Program Files\Python39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


 ARIMA(1,1,1)(1,0,1)[4] intercept   : AIC=-51.342, Time=0.18 sec
 ARIMA(0,1,0)(0,0,0)[4] intercept   : AIC=0.905, Time=0.01 sec
 ARIMA(1,1,0)(1,0,0)[4] intercept   : AIC=-41.752, Time=0.05 sec
 ARIMA(0,1,1)(0,0,1)[4] intercept   : AIC=-40.120, Time=0.07 sec
 ARIMA(0,1,0)(0,0,0)[4]             : AIC=15.157, Time=0.01 sec
 ARIMA(1,1,1)(0,0,1)[4] intercept   : AIC=-37.669, Time=0.16 sec
 ARIMA(1,1,1)(1,0,0)[4] intercept   : AIC=-43.050, Time=0.11 sec
 ARIMA(1,1,1)(2,0,1)[4] intercept   : AIC=-52.464, Time=0.23 sec
 ARIMA(1,1,1)(2,0,0)[4] intercept   : AIC=-54.401, Time=0.21 sec
 ARIMA(1,1,1)(3,0,0)[4] intercept   : AIC=-52.371, Time=0.23 sec
 ARIMA(1,1,1)(3,0,1)[4] intercept   : AIC=-51.062, Time=0.29 sec
 ARIMA(0,1,1)(2,0,0)[4] intercept   : AIC=-51.789, Time=0.10 sec
 ARIMA(1,1,0)(2,0,0)[4] intercept   : AIC=-45.681, Time=0.08 sec
 ARIMA(2,1,1)(2,0,0)[4] intercept   : AIC=-53.801, Time=0.31 sec
 ARIMA(1,1,2)(2,0,0)[4] intercept   : AIC=-53.253, Time=0.22 sec
 ARIMA(0,1,0)(2,0,0)[4] inte